# Домашнее задание к лекции "Базовые понятия статистики"

In [1]:
import pandas as pd

def load_data():
    df = pd.read_csv('horse_data.csv', names=range(1,29), na_values=['?'])
    return df[[1,2,4,5,9,12,13,23]].rename(columns={
        1: 'surgery',
        2: 'age',
        4: 'temp',
        5: 'pulse',
        9: 'mucous',
        12: 'peristalsis',
        13: 'abdominal',
        23: 'outcome'
    })

df = load_data()
print(df.describe())
print(df.mode())

          surgery         age        temp       pulse      mucous  \
count  299.000000  300.000000  240.000000  276.000000  253.000000   
mean     1.397993    1.640000   38.167917   71.913043    2.853755   
std      0.490305    2.173972    0.732289   28.630557    1.620294   
min      1.000000    1.000000   35.400000   30.000000    1.000000   
25%      1.000000    1.000000   37.800000   48.000000    1.000000   
50%      1.000000    1.000000   38.200000   64.000000    3.000000   
75%      2.000000    1.000000   38.500000   88.000000    4.000000   
max      2.000000    9.000000   40.800000  184.000000    6.000000   

       peristalsis   abdominal     outcome  
count   256.000000  244.000000  299.000000  
mean      2.917969    2.266393    1.551839  
std       0.976744    1.065131    0.737187  
min       1.000000    1.000000    1.000000  
25%       3.000000    1.000000    1.000000  
50%       3.000000    2.000000    1.000000  
75%       4.000000    3.000000    2.000000  
max       4.000000

В выбранных данных присутствует две числовые колонки, temp и pulse. Судя по разнице между средней и более устойчивой к выбросам медианной, в колонке pulse с большой вероятностью могут быть выбросы.
Судя по значениям колонки age, присутствуют ошибки. Разбираемся:

In [2]:
df.age.value_counts()

1    276
9     24
Name: age, dtype: int64

Очевидно, в этой колонке вместо 2 указано 9, исправляем. Так же, в колонке outcome, меняем значение 3 на 2, для целей аналитики разница между смертью и эвтаназией не существенна.

In [3]:
df.age.replace({9: 2}, inplace = True)
df.outcome.replace({3: 2}, inplace = True)

Ищем выбросы в числовых колонках temp, pulse:

In [4]:
def find_outliers(df, columns):
    for column in columns:
        q1 = df[column].quantile(.25)
        q3 = df[column].quantile(.75)
        iqr = q3 - q1
        lower_bound = q1 - (iqr * 1.5)
        upper_bound = q3 + (iqr * 1.5)
        remove_outliers = df[df[column].between(lower_bound, upper_bound, inclusive = True)].sort_values(column)
        print(pd.concat([df, remove_outliers]).drop_duplicates(keep=False).sort_values(column)[column])
        
find_outliers(df, ['temp', 'pulse'])

44     35.4
141    36.0
238    36.1
80     36.4
118    36.5
298    36.5
251    36.6
99     39.6
75     39.7
20     39.9
281    40.0
259    40.8
5       NaN
7       NaN
8       NaN
16      NaN
35      NaN
40      NaN
43      NaN
45      NaN
52      NaN
57      NaN
58      NaN
65      NaN
68      NaN
74      NaN
86      NaN
87      NaN
93      NaN
96      NaN
101     NaN
105     NaN
112     NaN
115     NaN
134     NaN
142     NaN
145     NaN
159     NaN
167     NaN
169     NaN
173     NaN
174     NaN
177     NaN
178     NaN
191     NaN
193     NaN
216     NaN
219     NaN
222     NaN
227     NaN
242     NaN
253     NaN
254     NaN
265     NaN
274     NaN
293     NaN
295     NaN
Name: temp, dtype: float64
41     150.0
275    150.0
55     160.0
3      164.0
255    184.0
5        NaN
52       NaN
58       NaN
74       NaN
93       NaN
115      NaN
117      NaN
126      NaN
151      NaN
159      NaN
174      NaN
204      NaN
216      NaN
227      NaN
Name: pulse, dtype: float64


Выбросы есть в каждой числовой колонке, но, если верить гуглу, такие значения вполне могут иметь место для больных лошадей. Оставляем выбросы как есть

В колонке 'age' пропусков нет, а ошибки мы уже поправили, а колонках 'surgery' и 'outcome' по 1 пропуску, поэтому строки с пропусками можно смело удалять:

In [5]:
df.dropna(subset=['surgery', 'outcome'], inplace=True)

Осталось 5 колонок с пропусками, которые надо заполнить: temp,pulse,mucous,peristalsis,abdominal
Попробуем сначала посмотреть зависимость числовых колонок temp и pulse от каждой из качественных колонок:

In [6]:
print(df.groupby('surgery').mean())
print(df.groupby('age').mean())
print(df.groupby('mucous').mean())
print(df.groupby('abdominal').mean())
print(df.groupby('outcome').mean())
print(df.groupby('peristalsis').mean())

             age       temp      pulse    mucous  peristalsis  abdominal  \
surgery                                                                    
1.0      1.10000  38.143571  76.396341  3.108108     3.141935   2.492958   
2.0      1.05042  38.204040  65.504505  2.490385     2.570000   1.960396   

          outcome  
surgery            
1.0      1.472222  
2.0      1.302521  
      surgery       temp       pulse    mucous  peristalsis  abdominal  \
age                                                                      
1    1.410909  38.125114   67.533597  2.855932     2.928270   2.245614   
2    1.250000  38.645000  123.363636  2.812500     2.777778   2.666667   

      outcome  
age            
1    1.396364  
2    1.500000  
         surgery       age       temp       pulse  peristalsis  abdominal  \
mucous                                                                      
1.0     1.544304  1.050633  38.216438   56.766234     2.351351   1.671233   
2.0     1.433333  1.200

Вижу и предполагаю следующие закономерности:
Колонка temp зависит от колонки age, сгруппируем и заполним медианой.
Колонка pulse от колонок age,abdomial,outcome, сгруппируем и заполним медианой.
Колонка peristalsis зависит от колонки age,abdomial,outcome, сгруппируем и заполним медианой.
Колонка abdominal зависит от колонки age,peristalsis,abdominal сгруппируем и заполним медианой.
Колонка mucous зависит от колонок abdomial,outcome,peristalsis, сгруппируем и заполним медианой.

In [7]:
df['temp'].fillna(df.groupby(['age'])['temp'].transform('median'), inplace=True)
df['pulse'].fillna(df.groupby(['age','abdominal','outcome'])['pulse'].transform('median'), inplace=True)
df['peristalsis'].fillna(df.groupby(['age','abdominal','outcome'])['peristalsis'].transform('median'), inplace=True)
df['abdominal'].fillna(df.groupby(['age','peristalsis','abdominal'])['abdominal'].transform('median'), inplace=True)
df['mucous'].fillna(df.groupby(['abdominal','outcome','peristalsis'])['mucous'].transform('median'), inplace=True)

In [8]:
print(df.describe())
print(df.mode())

          surgery         age        temp       pulse      mucous  \
count  299.000000  299.000000  299.000000  290.000000  266.000000   
mean     1.397993    1.080268   38.161538   72.055172    2.847744   
std      0.490305    0.272162    0.658266   28.170337    1.603750   
min      1.000000    1.000000   35.400000   30.000000    1.000000   
25%      1.000000    1.000000   37.900000   48.000000    1.000000   
50%      1.000000    1.000000   38.100000   64.000000    3.000000   
75%      2.000000    1.000000   38.500000   88.000000    4.000000   
max      2.000000    2.000000   40.800000  184.000000    6.000000   

       peristalsis   abdominal     outcome  
count   265.000000  243.000000  299.000000  
mean      2.935849    2.271605    1.404682  
std       0.969046    1.064207    0.491653  
min       1.000000    1.000000    1.000000  
25%       3.000000    1.000000    1.000000  
50%       3.000000    2.000000    1.000000  
75%       4.000000    3.000000    2.000000  
max       4.000000

In [ ]:
Метрики не сильно отклонились от стартовых, значит пропуски, с большой вероятность, заполнены верно